# embed_cloud_passages
---


In [1]:
import os
import json
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_postgres import PGVector
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [2]:
emb = HuggingFaceEmbeddings(
    model_name="Snowflake/snowflake-arctic-embed-l-v2.0"
)

load_dotenv("../.pg_env")
connection_string = os.getenv("PG_CONNECTION_STRING")

vectorstore = PGVector(
    connection=connection_string,
    embeddings=emb
)

In [3]:
import zipfile
import os

zip_path = "../corpora/passage_level/cloud.jsonl (1).zip"
extract_dir = "../corpora/passage_level/"

target_file = os.path.join(extract_dir, "cloud.jsonl")

if not os.path.exists(target_file):
    print("Extracting cloud.jsonl (1).zip...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_dir)
    print("✓ Extracted!")
else:
    print("cloud.jsonl already exists, skipping unzip.")


cloud.jsonl already exists, skipping unzip.


In [4]:
jsonl_path = target_file  

with open(target_file, "r", encoding="utf-8") as f:
    for line in tqdm(f):
        obj = json.loads(line)

        doc = Document(
                page_content=obj["text"],
                metadata={
                    "doc_id": obj["id"],
                    "title": obj.get("title", ""),
                    "url": obj.get("url", "")
                }
            )
        vectorstore.add_documents([doc])

72442it [4:52:17,  4.13it/s]


## Retreival Test
---

In [5]:
def retreive_top_k_documents_for_query(query, k=3):
    docs_and_scores = vectorstore.similarity_search_with_score(
        query,
        k=k
    )

    for doc, score in docs_and_scores:
        print(doc.metadata["doc_id"], "→", score, "→", doc.page_content, "\n\n\n\n\n\n\n")

In [6]:
retreive_top_k_documents_for_query("how to add web chat widget?", 1)

ibmcld_16295-7-1721 → 0.2773657821823886 → 
Embedding the web chat on your page 

To add the web chat widget to your website, all you need to do is embed a generated script element in your HTML source.

The web chat integration is automatically included for every assistant, and is configured separately for each environment.

To add the web chat to your website, follow these steps:



1. On the ![Integrations icon](https://cloud.ibm.com/docs-content/v1/content/ca9232b3702768f24bb897f48f10778fcda28718/watson-assistant/images/integrations-icon.png)Integrations page, find the Web chat tile and click click Open. The Open web chat window opens.
2. In the Environment field, select the environment you want the web chat widget to connect to. Click Confirm.

The Web chat page opens, showing the settings for the web chat integration in the selected environment.

The preview pane shows what the web chat will look like when it is embedded in a web page. If you see a message that starts with, There 

In [7]:
retreive_top_k_documents_for_query("Can I extend the web chat? if so, how can I do that?", 1)

ibmcld_16384-1889-3334 → 0.36289986150859843 → 
You can also use the web chat integration settings in the Watson Assistant user interface to configure the web chat for your website and your customers.

If you are a developer, you can further customize and extend the web chat by writing code and using the web chat API. You can also use a WebView with a JavaScript bridge to add the web chat to your mobile app.

The following documentation topics provide more information about the capabilities of the web chat integration, how to configure and deploy it, and how to customize it.



* [How the web chat works](https://cloud.ibm.com/docs/watson-assistant?topic=watson-assistant-web-chat-architecture): Overview of web chat capabilities and architecture
* [Embedding the web chat on your page](https://cloud.ibm.com/docs/watson-assistant?topic=watson-assistant-deploy-web-chat): How to embed the web chat widget on your website
* [Web chat setup](https://cloud.ibm.com/docs/watson-assistant?topic=wat